## Advanced Tiling Strategies using DeGirum PySDK
This notebook is an example of how to perform advanced tiling using DeGirum PySDK on Hailo hardware. The advanced tiling
strategies are used to mitigate partial/duplicate/overlapping objects introduced by tiling an image for
object detection. Four different detection merging strategies are demonstrated.

This script works with the following inference options:

1. Run inference on DeGirum Hailo accelerator directly installed on your computer;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum Cloud Platform.

To try different options, you need to specify the appropriate `hw_location` option.

You can change `image_source` to a URL or path to another image file.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

# Overview of tiling strategies
degirum_tools comes with four different tiling strategies. The base TileModel simply recombines all detections from each tile and subsequently performs NMS. The LocalGlobalTileModel performs an inference on all tiles and the whole image and selects detections from the tiles or the whole image based on a large object threshold. The BoxFusionTileModel fuses split detections that are detected on the edges of tiles based on a one dimensional IoU threshold. The BoxFusionLocalGlobalTileModel combines the former two strategies.

The base tiling strategy improves the mAP of small objects at the expense of large objects. By incorporating the LocalGlobal strategy, it is possible to recapture the mAP lost from tiling. The BoxFusion strategy gives modest gains in mAP across all object sizes due to relatively fewer detections occuring on edges/corners of tiles.

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local hailo device
#     IP address for AI server inference
# image_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to image file
# model_name: name of the model for running AI inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# class_set: whitelist for classes to detect
# yolo11n_visdrone_person--640x640_quant_hailort_multidevice_1
# /yolo11n_visdrone_person--640x640_quant_hailort_multidevice_1
inference_host_address = "@local"
zoo_url = "degirum/models_hailort"
token = ""
model_name = 'yolo11n_visdrone_person--640x640_quant_hailort_multidevice_1'
image_source = 'assets/drone_pedestrain.jpg'
class_set = {"pedestrian", "people"}

#### The rest of the cells below should run without any modifications

In [ ]:
# imports and variables used in most cells
import degirum as dg
import degirum_tools as dgt

from degirum_tools.tile_compound_models import TileExtractorPseudoModel, TileModel, LocalGlobalTileModel, BoxFusionTileModel, BoxFusionLocalGlobalTileModel
from degirum_tools import NmsBoxSelectionPolicy, NmsOptions

# Base NMS options.
nms_options = NmsOptions(
    threshold=0.6,
    use_iou=True,
    box_select=NmsBoxSelectionPolicy.MOST_PROBABLE,
)

## No tiling example

In [ ]:
# Load model to be used for tiling
# load AI model
model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    image_backend='pil')

model.output_class_set = class_set  # filter class outputs

results = model(image_source)
results.image_overlay

The following examples all are tiled with 3 columns, 2 rows, and a 10% overlap minimum between each tile.

## Base TileModel example

In [ ]:
tile_extractor = TileExtractorPseudoModel(cols=3,
                                          rows=2,
                                          overlap_percent=0.1,
                                          model2=model,
                                          global_tile=False)
tile_model = TileModel(model1=tile_extractor,
                       model2=model,
                       nms_options=nms_options)
results = tile_model(image_source)
results.image_overlay

## LocalGlobalTileModel example

In [ ]:
tile_extractor = TileExtractorPseudoModel(cols=3,
                                          rows=2,
                                          overlap_percent=0.1,
                                          model2=model,
                                          global_tile=True)
tile_model = LocalGlobalTileModel(model1=tile_extractor,
                                  model2=model,
                                  large_object_threshold=0.01,
                                  nms_options=nms_options)
results = tile_model(image_source)
results.image_overlay

## BoxFusionTileModel example

In [ ]:
tile_extractor = TileExtractorPseudoModel(cols=3,
                                          rows=2,
                                          overlap_percent=0.1,
                                          model2=model,
                                          global_tile=False)
tile_model = BoxFusionTileModel(model1=tile_extractor,
                                model2=model,
                                edge_threshold=0.02,
                                fusion_threshold=0.8)
results = tile_model(image_source)
results.image_overlay

## BoxFusionLocalGlobalTileModel example

In [ ]:
tile_extractor = TileExtractorPseudoModel(cols=3,
                                          rows=2,
                                          overlap_percent=0.1,
                                          model2=model,
                                          global_tile=True)
tile_model = BoxFusionLocalGlobalTileModel(model1=tile_extractor,
                                           model2=model,
                                           large_object_threshold=0.01,
                                           edge_threshold=0.02,
                                           fusion_threshold=0.8,
                                           nms_options=nms_options)
results = tile_model(image_source)
results.image_overlay